In [6]:
import ultrasoundgeneration
import constants
import numpy as np

whoop


In [2]:
img = ultrasoundgeneration.load_image_annotation?

In [ ]:
img = ultrasoundgeneration.load_image_annotation

In [ ]:
img = ultrasoundgeneration.load_image_annotation

In [15]:
img = ultrasoundgeneration.load_image_annotation

In [13]:
img = ultrasoundgeneration.load_image_annotation(ultrasoundgeneration.test_names[0], ultrasoundgeneration.test_volumes_path)

In [1]:
np.array(img[1].GetLargestPossibleRegion().GetSize())

NameError: name 'np' is not defined

In [18]:
import tensorflow.keras as keras

In [6]:
import train_network_withangle

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 4)  0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 128, 128, 1)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 128, 128, 1)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 128, 128, 1)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_3 (

In [7]:
train_network_withangle.model.load_weights("models/includeangle")

In [4]:
import ultrasoundgeneration

In [ ]:
############## sample ultrasound path



In [8]:
import itk
import os
def loadFile(name):
    imageType = itk.Image[itk.F, 3]
    
    readerType = itk.ImageFileReader[imageType]
    
    reader = readerType.New()
    
    reader.SetFileName(name)
    
    im = reader.GetOutput()
    
    
    reader.Update()
    print(im.GetSpacing())
    return np.array(itk.PyBuffer[imageType].GetArrayFromImage(im))
import xml.etree.ElementTree 
def getTransforms(filename):
    
 
    with open(filename, 'rb') as f:
        lines = f.readlines()
    from collections import defaultdict
    fields = defaultdict(lambda: [])
    for entry in [str(l)[11:].split(" ") for l in lines if l[:3] == b'Seq']:
        frame, kind = entry[0].split("_")
        fields[kind].append(entry[1:])



    
    ImageToReference = fields['ImageToReferenceTransform']
    ImageToReference = np.array(ImageToReference)

    ImageToReference = ImageToReference[:, 1:-1].reshape(-1, 4, 4)

    ImageToReference = np.array(ImageToReference, dtype=np.float64)

    return ImageToReference
name = os.path.join(constants.volumes_path, "HultrasoundI2R/h-20180315_120244_ImageToReference.mha")
transforms = getTransforms(name)

In [10]:
transforms.shape

(598, 4, 4)

In [8]:
import ultrasoundgeneration as ug

img, annotation = ug.load_image_annotation(
    ug.test_names[0], ug.test_volumes_path)

In [9]:
probe = ug.TwoSensorProbe(20, -np.pi /4)

In [19]:
def generate_sample(image, origin1, origin2, direction1, direction2):
    probe = ug.TwoSensorProbe(20, -np.pi /4)
    a, b = ug.slice_multiprobe(image, probe, origin1, direction1)
    c, d = ug.slice_multiprobe(image, probe, origin2, direction2)
    rotation = np.dot(np.linalg.inv(direction1), direction2) 
    movement = np.transpose(np.dot(np.linalg.inv(direction1), np.transpose(origin2 - origin1)))
    return {"data": [a, b, c, d], "classes": [movement, rotation]}

    

In [20]:
slice_idx = 65
direction = np.array([[0, 0, 1],
                  [-1, 0, 0],
                  [0, -1, 0]], dtype=np.float64)
origin1 = np.array([annotation[1][slice_idx], -annotation[0][slice_idx], -slice_idx])
slice_idx += 1
origin2 = np.array([annotation[1][slice_idx], -annotation[0][slice_idx], -slice_idx])

sample = generate_sample(img, origin1, origin2, direction, direction)

In [21]:
def convert_dataset(dataset, simulate_ultrasound=False):
    data = []
    classes = []
    
    for elem in dataset:
        if not simulate_ultrasound:
            data_entry = np.array(np.stack(elem["data"], axis=-1), dtype=np.float32)
            data_entry += 1000
            data_entry /= 2000
        else: 
            data_entry = [approx_ultrasound(d) for d in elem["data"]]
            data_entry = np.stack(data_entry, axis=-1) / 10 + 1
            data_entry = np.nan_to_num(data_entry)
            data_entry = np.clip(data_entry, -5, 5)

        
        data.append(data_entry)

        class_entry = elem["classes"]
        class_entry = np.concatenate([class_entry[0] / 4, class_entry[1].flatten() * 40])
        class_entry[[3, 7, 11]] -= 40
        classes.append(class_entry)
    data = np.array(data)
    classes = np.array(classes)
    return data, classes

In [23]:
data, classes = convert_dataset([sample])

In [24]:
classes

array([[0.20507812, 0.25      , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ]])

In [3]:
from julia.api import Julia
import julia
jl = Julia(compiled_modules=False)

JuliaError: Exception 'ArgumentError' occurred while calling julia code:
using PyCall

In [4]:
julia.__version__

'0.5.3'